In [1]:
from pathlib import Path
_API_DIR = Path("/Users/joregan/riksdag/riksdag-api-out")

In [22]:
def endswith_list(text, items):
    for it in items:
        if text.endswith(it):
            return True
    return False

In [27]:
import json
def json_matches_years(filename, years):
    ret_val = False
    with open(filename) as f:
        data = json.load(f)
        if not "videodata" in data:
            raise Exception(f"File {filename} missing key 'videodata'")
        videodata = data["videodata"]
        for vdata in videodata:
            if "debatedate" in vdata and vdata["debatedate"] is not None and vdata["debatedate"] != "":
                date = vdata["debatedate"]
                if endswith_list(date.strip(), years):
                    return True
    return False
                
json_matches_years("/Users/joregan/riksdag/riksdag-api-out/H9C420210914zz2", ["2017", "2018"])

False

In [26]:
matches = []
for file in _API_DIR.glob("H*"):
    if json_matches_years(file, ["2017", "2018"]):
        matches.append(str(file))

AttributeError: 'NoneType' object has no attribute 'strip'